In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import requests
import re
import os
import time

# Scraping Stadium Capacity Data From Wikipedia

### Website (https://en.wikipedia.org/wiki/List_of_current_NFL_stadiums)


In [62]:
url = 'https://en.wikipedia.org/wiki/List_of_current_NFL_stadiums'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [63]:
#Scraping Column Names from 'Th'
table = soup.find_all('table', class_ = "wikitable sortable plainrowheaders")[0]
titles = table.find_all('th')
table_titles = [title.text.strip() for title in titles]
column_names = table_titles[0:8]

In [64]:
stadium_names = pd.DataFrame(table_titles[9:39], columns = ["Name"])
stadium_names.head()

,Name
0,Acrisure Stadium
1,Allegiant Stadium
2,Arrowhead Stadium
3,AT&T Stadium
4,Bank of America Stadium


In [65]:
column_data = table.find_all('tr')

stadium_info_list = []

for row in column_data[1:]:
    row_data = row.find_all('td')[1:7]
    individual_row_data = [data.text.strip() for data in row_data]
    stadium_info_list.append(individual_row_data)

In [66]:
stadium_info = pd.DataFrame(stadium_info_list, columns = column_names[2:])

stadium_info_df = pd.concat([stadium_names, stadium_info], axis = 1)
stadium_info_df.head()

,Name,Team(s),Location,Capacity,Surface,Roof type,Opened
0,Acrisure Stadium,Pittsburgh Steelers,"Pittsburgh, Pennsylvania","68,400",Kentucky bluegrass,Open,2001
1,Allegiant Stadium,Las Vegas Raiders,"Paradise, Nevada","65,000",Bermuda grass,Fixed,2020
2,Arrowhead Stadium,Kansas City Chiefs,"Kansas City, Missouri","76,416",Bermuda grass,Open,1972
3,AT&T Stadium,Dallas Cowboys,"Arlington, Texas","80,000",Hellas Matrix Turf,Retractable,2009
4,Bank of America Stadium,Carolina Panthers,"Charlotte, North Carolina","74,867",FieldTurf,Open,1996


In [67]:
#Creating Additional Rows for Shared Stadiums
shares_stadium = (stadium_info_df["Name"] == "MetLife Stadium") | (stadium_info_df["Name"] == "SoFi Stadium")
shared_stadiums = stadium_info_df[shares_stadium]
shared_stadiums

,Name,Team(s),Location,Capacity,Surface,Roof type,Opened
20,MetLife Stadium,New York GiantsNew York Jets,"East Rutherford, New Jersey","82,500",FieldTurf Core[32],Open,2010
26,SoFi Stadium,Los Angeles RamsLos Angeles Chargers,"Inglewood, California","70,240",Hellas Matrix Turf,Fixed,2020


# Cleaning Stadium DataFrame


In [68]:
stadium_info_df = pd.concat([stadium_info_df, shared_stadiums])
stadium_info_df.sort_index(inplace = True)
stadium_info_df.reset_index(drop = True, inplace = True)

# #Removing Commas from Values in Capacity
stadium_info_df["Capacity"] = stadium_info_df["Capacity"].replace(",","", regex = True)
# #Changing Capacity Column to Integer Type
stadium_info_df["Capacity"] = stadium_info_df["Capacity"].astype(float)
stadium_info_df.head()

,Name,Team(s),Location,Capacity,Surface,Roof type,Opened
0,Acrisure Stadium,Pittsburgh Steelers,"Pittsburgh, Pennsylvania",68400.0,Kentucky bluegrass,Open,2001
1,Allegiant Stadium,Las Vegas Raiders,"Paradise, Nevada",65000.0,Bermuda grass,Fixed,2020
2,Arrowhead Stadium,Kansas City Chiefs,"Kansas City, Missouri",76416.0,Bermuda grass,Open,1972
3,AT&T Stadium,Dallas Cowboys,"Arlington, Texas",80000.0,Hellas Matrix Turf,Retractable,2009
4,Bank of America Stadium,Carolina Panthers,"Charlotte, North Carolina",74867.0,FieldTurf,Open,1996


### Changing Team Name Column for Shared Stadiums

In [69]:
# Renaming "Team(s)" column as "Team"
stadium_info_df.rename(columns = {"Team(s)": "Team"}, inplace = True)


stadium_info_df.iloc[20, 1] = "New York Jets"
stadium_info_df.iloc[21, 1] = "New York Giants"

stadium_info_df.iloc[27, 1] = "Los Angeles Chargers"
stadium_info_df.iloc[28, 1] = "Los Angeles Rams"

stadium_info_df

,Name,Team,Location,Capacity,Surface,Roof type,Opened
0,Acrisure Stadium,Pittsburgh Steelers,"Pittsburgh, Pennsylvania",68400.0,Kentucky bluegrass,Open,2001
1,Allegiant Stadium,Las Vegas Raiders,"Paradise, Nevada",65000.0,Bermuda grass,Fixed,2020
2,Arrowhead Stadium,Kansas City Chiefs,"Kansas City, Missouri",76416.0,Bermuda grass,Open,1972
3,AT&T Stadium,Dallas Cowboys,"Arlington, Texas",80000.0,Hellas Matrix Turf,Retractable,2009
4,Bank of America Stadium,Carolina Panthers,"Charlotte, North Carolina",74867.0,FieldTurf,Open,1996
5,Caesars Superdome,New Orleans Saints,"New Orleans, Louisiana",73208.0,FieldTurf Revolution 360[10],Fixed,1975
6,Huntington Bank Field,Cleveland Browns,"Cleveland, Ohio",67431.0,Kentucky bluegrass,Open,1999
7,Empower Field at Mile High,Denver Broncos,"Denver, Colorado",76125.0,Kentucky bluegrass,Open,2001
8,EverBank Stadium,Jacksonville Jaguars,"Jacksonville, Florida",67838.0,Bermuda grass,Open,1995
9,Ford Field,Detroit Lions,"Detroit, Michigan",65000.0,FieldTurf CORE[16],Fixed,2002


### Reducing DF Columns

In [70]:
stadium_info_df_clean = stadium_info_df[["Team", "Name", "Capacity", "Opened"]]
stadium_info_df_clean.head()

,Team,Name,Capacity,Opened
0,Pittsburgh Steelers,Acrisure Stadium,68400.0,2001
1,Las Vegas Raiders,Allegiant Stadium,65000.0,2020
2,Kansas City Chiefs,Arrowhead Stadium,76416.0,1972
3,Dallas Cowboys,AT&T Stadium,80000.0,2009
4,Carolina Panthers,Bank of America Stadium,74867.0,1996


## Adjusting Maximum Capacities to Account for Standing Room

In [71]:
stadium_info_df_clean.loc[stadium_info_df_clean["Team"] == "Cincinnati Bengals", "Capacity"] = 67260
stadium_info_df_clean.loc[stadium_info_df_clean["Team"] == "Cleveland Browns", "Capacity"] = 73718

stadium_info_df_clean.loc[stadium_info_df_clean["Team"] == "Dallas Cowboys", "Capacity"] = 90000

stadium_info_df_clean.sort_values(by = "Team")

,Team,Name,Capacity,Opened
30,Arizona Cardinals,State Farm Stadium,63400.0,2006
19,Atlanta Falcons,Mercedes-Benz Stadium‡,71000.0,2017
18,Baltimore Ravens,M&T Bank Stadium,71008.0,1998
12,Buffalo Bills,Highmark Stadium,71608.0,1973
4,Carolina Panthers,Bank of America Stadium,74867.0,1996
29,Chicago Bears,Soldier Field,61500.0,1924[a]
25,Cincinnati Bengals,Paycor Stadium,67260.0,2000
6,Cleveland Browns,Huntington Bank Field,73718.0,1999
3,Dallas Cowboys,AT&T Stadium,90000.0,2009
7,Denver Broncos,Empower Field at Mile High,76125.0,2001


# Scraping Team Abbreviation Table

In [72]:
url = "https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Football_League/National_Football_League_team_abbreviations"

# Read the table directly from the URL using pandas read_html
tables = pd.read_html(url)

abbreviations_df = tables[0]
abbreviations_df.columns = abbreviations_df.iloc[0]
abbreviations_df.rename(columns = {'Franchise':'Team', 'Commonly Used Abbreviations':'Tm'}, inplace = True)
abbreviations_df.drop(index = 0, inplace = True)
abbreviations_df.drop(columns = 'Official Team Abbreviation Codes', inplace = True)

#Changing Jaguars abbreviation from JAC to JAX
abbreviations_df.loc[abbreviations_df["Team"] == "Jacksonville Jaguars", "Tm"] = "JAX"
abbreviations_df.head()

,Team,Tm
1,Arizona Cardinals,ARI
2,Atlanta Falcons,ATL
3,Baltimore Ravens,BAL
4,Buffalo Bills,BUF
5,Carolina Panthers,CAR


# Scraping Attendance Data (2013-2023)

In [73]:
base_url = "https://www.pro-football-reference.com/years/{}/attendance.htm"

# Create a list of URLs for the years 2013 to 2023
years = [year for year in range(2013,2024) if year != 2020]
urls = [base_url.format(year) for year in years]

def fetch_attendance_data(url):
    """
    Fetches attendance data from the given URL and returns a DataFrame.
    
    Parameters:
    url (str): The URL of the webpage containing attendance data.

    Returns:
    pd.DataFrame: A DataFrame containing the parsed attendance data.
    """
    # Fetch the web page
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the specific table
    table = soup.find('table', {'id': 'attendance'})
    if table is None:
        raise ValueError("Attendance table not found")

    # Extract headers
    headers = [th.text for th in table.find('thead').find_all('th')]

    # Extract rows
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        cells = tr.find_all(['th', 'td'])
        row = [cell.text.strip() for cell in cells]  # Remove any extra whitespace
        rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=headers)

    # Remove any unwanted rows or columns (optional)
    df = df.dropna(how='all')  # Drop rows where all elements are NaN

    # Remove spaces from column names
    df.columns = df.columns.str.replace(' ', '')

    #Rename Team column to "Team"
    df.rename(columns = {"Tm":"Team"}, inplace = True)
    
    return df

# Dictionary to hold attendance data for each year
attendance_dfs = {}

# Loop to fetch data and create DataFrames for each year
for year, url in zip(years, urls):
    try:
        # Fetch data
        df = fetch_attendance_data(url)
        attendance_dfs[year] = df
        print(f"Successfully fetched data for {year}")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred for {year}: {http_err}")
    except Exception as e:
        print(f"An error occurred for {year}: {e}")

# Access and display data for 2013
attendance_dfs[2023].head()

Successfully fetched data for 2013
Successfully fetched data for 2014
Successfully fetched data for 2015
Successfully fetched data for 2016
Successfully fetched data for 2017
Successfully fetched data for 2018
Successfully fetched data for 2019
Successfully fetched data for 2021
Successfully fetched data for 2022
Successfully fetched data for 2023


,Team,Total,Home,Away,Week1,Week2,Week3,Week4,Week5,Week6,...,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,Arizona Cardinals,"1,113,568","502,912","610,656","64,693","62,080","62,915","71,521","62,651","71,202",...,"67,919","62,861","70,247","62,177","66,436",Bye,"63,967","59,978","69,879","63,197"
1,Atlanta Falcons,"1,176,416","556,828","619,588","69,597","69,251","63,803","85,716","69,461","69,921",...,"69,715","62,861",Bye,"70,166","74,455","69,496","70,301","69,221","61,752","70,072"
2,Baltimore Ravens,"1,170,734","635,380","535,354","70,136","66,015","70,038","67,919","67,272","61,011",...,"70,753","70,424","70,997","70,240",Bye,"70,492","68,021","71,812","71,196","70,355"
3,Buffalo Bills,"1,183,200","626,483","556,717","83,345","70,772","64,291","70,881","61,273","70,515",...,"66,965","70,318","70,603","69,879",Bye,"73,639","70,933","67,678","70,772","66,292"
4,Carolina Panthers,"1,163,926","573,086","590,840","69,597","72,709","68,699","72,842","63,648","65,529",...,"72,301","61,200","73,543","62,493","62,432","70,009","70,301","70,875","67,233","69,010"


# Cleaning Attendance Data

In [74]:
# Dropping Unecessary columns
for year in range(2013,2024):
    if year != 2020:
        attendance_dfs[year].drop(columns = ['Total', 'Home', 'Away'], inplace = True)

In [75]:
# Adjusting Inconsistent Team Names
for year in range(2013,2024):
    if year != 2020:
        attendance_dfs[year] = attendance_dfs[year].map(lambda x: 'Los Angeles Chargers' if 'Chargers' in x else x)
        attendance_dfs[year] = attendance_dfs[year].map(lambda x: 'Los Angeles Rams' if 'Rams' in x else x)
        attendance_dfs[year] = attendance_dfs[year].map(lambda x: 'Washington Commanders' if 'Washington' in x else x)
        attendance_dfs[year] = attendance_dfs[year].map(lambda x: 'Las Vegas Raiders' if 'Raiders' in x else x)    

In [76]:
# Changing Tm column to Team abbreviations
for year in range(2013,2024):
    if year != 2020:
        attendance_dfs[year] = pd.merge(abbreviations_df, attendance_dfs[year], on = 'Team', how = 'right')
        attendance_dfs[year].drop(columns = 'Team', inplace = True)
        
        #Sorting Teams Alphabetically to Match up with Schedule DF
        attendance_dfs[year].sort_values(by = "Tm", inplace = True)
        attendance_dfs[year].reset_index(drop = True, inplace = True)

attendance_dfs[2023].head()

,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,ARI,"64,693","62,080","62,915","71,521","62,651","71,202","68,781","63,064","67,919","62,861","70,247","62,177","66,436",Bye,"63,967","59,978","69,879","63,197"
1,ATL,"69,597","69,251","63,803","85,716","69,461","69,921","62,620","68,008","69,715","62,861",Bye,"70,166","74,455","69,496","70,301","69,221","61,752","70,072"
2,BAL,"70,136","66,015","70,038","67,919","67,272","61,011","70,989","63,064","70,753","70,424","70,997","70,240",Bye,"70,492","68,021","71,812","71,196","70,355"
3,BUF,"83,345","70,772","64,291","70,881","61,273","70,515","64,628","70,416","66,965","70,318","70,603","69,879",Bye,"73,639","70,933","67,678","70,772","66,292"
4,CAR,"69,597","72,709","68,699","72,842","63,648","65,529",Bye,"71,505","72,301","61,200","73,543","62,493","62,432","70,009","70,301","70,875","67,233","69,010"


## Missing Values

In [77]:
#Searching for Missing Values
for year in range(2013, 2024):
    if year != 2020:
        if (attendance_dfs[year] == "").sum().sum() > 0:
            print(year)

2021


In [78]:
# Inputting Missing Values
attendance_dfs[2021].loc[attendance_dfs[2021]["Tm"] == "CLE", "Week15"] = 67431
attendance_dfs[2021].loc[attendance_dfs[2021]["Tm"] == "LV", "Week15"] = 67431
attendance_dfs[2021].loc[attendance_dfs[2021]["Tm"] == "BUF", "Week14"] = 65655
attendance_dfs[2021].loc[attendance_dfs[2021]["Tm"] == "TB", "Week14"] = 65655

(attendance_dfs[2021] == "").sum().sum()

0

In [79]:
# Changing Data Type from str to int
for year in range(2013,2024):
    if year != 2020:
        #Removing commas from attendance value counts
        attendance_dfs[year] = attendance_dfs[year].replace(',','', regex = True)
        #Removing Bye Weeks
        attendance_dfs[year] = attendance_dfs[year].replace('Bye','0', regex = True)
        #Fill Blank values with 0
        
        #Changing data type from string to int
        for col in attendance_dfs[year].columns[1:]:
            attendance_dfs[year][col] = attendance_dfs[year][col].astype(int)
attendance_dfs[2023].dtypes

Tm        object
Week1      int32
Week2      int32
Week3      int32
Week4      int32
Week5      int32
Week6      int32
Week7      int32
Week8      int32
Week9      int32
Week10     int32
Week11     int32
Week12     int32
Week13     int32
Week14     int32
Week15     int32
Week16     int32
Week17     int32
Week18     int32
dtype: object

## Writing Cleaned Attendance Data to CSV Files for Exploratory Data Analysis

In [278]:
folder_path = 'C:/Users/petew/NFL Attendance Project/Data Files/Attendance Data for EDA'

# Naming Excel File to write Data to
excel_filename = 'Cleaned_Attendance_Data_All.xlsx'
full_file_path = os.path.join(folder_path, excel_filename)

# Create an Excel writer with the custom filename and folder path
excel_writer = pd.ExcelWriter(full_file_path, engine='xlsxwriter')

for year, df in attendance_dfs.items():
    sheet_name = f"{year}"
    df.to_excel(excel_writer, sheet_name=sheet_name, index=False)


# Save and close the Excel writer
excel_writer.close()

In [142]:
os.getcwd() + "/Data Files/Attendance Data for EDA"

'C:\\Users\\petew\\NFL Attendance Project/Data Files/Attendance Data for EDA'

### Adding Capacity Column to Each Team Row

In [141]:
#Joining Stadium Info Dataframe with Abbreviation DataFrame
capacity_abbreviation_join = pd.merge(stadium_info_df[["Team", "Capacity"]], abbreviations_df, on = "Team", how = "right")
#Dropping "Team" column
capacity_abbreviation_df = capacity_abbreviation_join[["Capacity", "Tm"]]

#Creating New Dictionary for all Capacity Percentage Dataframes
capacity_dfs = {}

#Joining Capacity and Abbreviation DF with Attendance DF
for year in range(2013,2024):
    if year != 2020:
        capacity_dfs[year] = pd.merge(capacity_abbreviation_df, attendance_dfs[year], on = "Tm", how = "right")
    
capacity_dfs[2023].head()

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,63400.0,ARI,64693,62080,62915,71521,62651,71202,68781,63064,67919,62861,70247,62177,66436,0,63967,59978,69879,63197
1,71000.0,ATL,69597,69251,63803,85716,69461,69921,62620,68008,69715,62861,0,70166,74455,69496,70301,69221,61752,70072
2,71008.0,BAL,70136,66015,70038,67919,67272,61011,70989,63064,70753,70424,70997,70240,0,70492,68021,71812,71196,70355
3,71608.0,BUF,83345,70772,64291,70881,61273,70515,64628,70416,66965,70318,70603,69879,0,73639,70933,67678,70772,66292
4,74867.0,CAR,69597,72709,68699,72842,63648,65529,0,71505,72301,61200,73543,62493,62432,70009,70301,70875,67233,69010


In [142]:
for year in range(2013,2024):
    if year != 2020:
        capacity_dfs[year]["Capacity"] = capacity_dfs[year]["Capacity"].astype(int)

capacity_dfs[2023].head()

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,63400,ARI,64693,62080,62915,71521,62651,71202,68781,63064,67919,62861,70247,62177,66436,0,63967,59978,69879,63197
1,71000,ATL,69597,69251,63803,85716,69461,69921,62620,68008,69715,62861,0,70166,74455,69496,70301,69221,61752,70072
2,71008,BAL,70136,66015,70038,67919,67272,61011,70989,63064,70753,70424,70997,70240,0,70492,68021,71812,71196,70355
3,71608,BUF,83345,70772,64291,70881,61273,70515,64628,70416,66965,70318,70603,69879,0,73639,70933,67678,70772,66292
4,74867,CAR,69597,72709,68699,72842,63648,65529,0,71505,72301,61200,73543,62493,62432,70009,70301,70875,67233,69010


In [143]:
#Maximum Capacity Adjustments for Changed Home Stadiums

# Old Stadiums:
# 49ers, Candlestick Park, 2013, 69732
capacity_dfs[2013].loc[capacity_dfs[2013]["Tm"] == "SF", "Capacity"] = 69732

# Falcons, Georgia Dome, 2013-2016, 71228
for year in range(2013,2017):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "ATL", "Capacity"] = 71228

# Vikings, Humphrey Metrodome, 2013, 64121
capacity_dfs[2013].loc[capacity_dfs[2013]["Tm"] == "MIN", "Capacity"] = 64121

# Vikings, TCF Bank Stadium, 2014-2015, 52525
for year in range(2014,2016):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "MIN", "Capacity"] = 52525

# Raiders, Oakland Coliseum, 2013-2019, 56057
for year in range(2013,2020):
    if year != 2020:
        capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "LV", "Capacity"] = 56057

# Chargers, Qualcomm Stadium, 2013-2016, 70561
for year in range(2013,2017):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "LAC", "Capacity"] = 70561

# Chargers, StubHub Center, 2017-2020, 27000
for year in range(2017,2021):
    if year != 2020:
        capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "LAC", "Capacity"] = 27000

# Rams, Edward Jones Dome 2013-2015, 66000
for year in range(2013,2016):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "LAR", "Capacity"] = 66000

# Rams, Los Angeles Memorial Coliseum, 2016-2020, 93607
for year in range(2016,2021):
    if year != 2020:
        capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "LAR", "Capacity"] = 93607

#Washington Commanders Capacity Decrease: (2013-2015), 85000
for year in range(2013,2016):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "WAS", "Capacity"] = 85000
    
#Washington Commanders Capacity Decrease: (2016-2019), 82000
for year in range(2016,2020):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "WAS", "Capacity"] = 82000

#Washington Commanders Capacity Decrease: (2020-2023), 67717
for year in range(2021,2024):
    capacity_dfs[year].loc[capacity_dfs[year]["Tm"] == "WAS", "Capacity"] = 67717


# Reading in Schedule Data (Scraped in R file)

In [144]:
# Creating Dictionary to store schedule data frames
schedule_dfs = {}

#For Loop to read in all 10 files
for year in range(2013,2024):
    file_path = f"Data Files/Schedule Data/NFLScheduleMatrix{year}.csv"
    schedule_dfs[year] = pd.read_csv(file_path)
    schedule_dfs[year].sort_values(by = "Team", inplace = True)
    schedule_dfs[year].reset_index(drop = True, inplace = True)

schedule_dfs[2023].head()

,Team,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,ARI,@WSH,NYG,DAL,@SF,CIN,@LAR,@SEA,BAL,@CLE,ATL,@HOU,LAR,@PIT,BYE,SF,@CHI,@PHI,SEA
1,ATL,CAR,GB,@DET,@JAX,HOU,WSH,@TB,@TEN,MIN,@ARI,BYE,NO,@NYJ,TB,@CAR,IND,@CHI,@NO
2,BAL,HOU,@CIN,IND,@CLE,@PIT,@TEN,DET,@ARI,SEA,CLE,CIN,@LAC,BYE,LAR,@JAX,@SF,MIA,PIT
3,BUF,@NYJ,LV,@WSH,MIA,JAX,NYG,@NE,TB,@CIN,DEN,NYJ,@PHI,BYE,@KC,DAL,@LAC,NE,@MIA
4,CAR,@ATL,NO,@SEA,MIN,@DET,@MIA,BYE,HOU,IND,@CHI,DAL,@TEN,@TB,@NO,ATL,GB,@JAX,TB


## Cleaning Schedule Data

In [145]:
# Removing away games and BYE week values and replacing with blanks

for year in range(2013,2024):
    schedule_dfs[year] = schedule_dfs[year].map(lambda x: '' if '@' in x else '' if 'BYE' in x else x)

# Changing Team column to Tm
for year in range(2013,2024):
    schedule_dfs[year].rename(columns = {'Team':'Tm'}, inplace = True)

schedule_dfs[2023].head()

,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,ARI,,NYG,DAL,,CIN,,,BAL,,ATL,,LAR,,,SF,,,SEA
1,ATL,CAR,GB,,,HOU,WSH,,,MIN,,,NO,,TB,,IND,,
2,BAL,HOU,,IND,,,,DET,,SEA,CLE,CIN,,,LAR,,,MIA,PIT
3,BUF,,LV,,MIA,JAX,NYG,,TB,,DEN,NYJ,,,,DAL,,NE,
4,CAR,,NO,,MIN,,,,HOU,IND,,DAL,,,,ATL,GB,,TB


### Making Schedule Changes Due to Postponments(BUF 2014, NYJ 2014, MIA 2017)

In [146]:
#BUF 2014
#Omitting Week 12 Attendance because Game was played at Ford Field (Lions Stadium)
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "BUF", "Week12"] = ""
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "BUF", "Week13"] = "CLE"
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "BUF", "Week14"] = ""
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "BUF", "Week15"] = "GB"
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "BUF", "Week16"] = ""

#MIA 2017

schedule_dfs[2017].loc[schedule_dfs[2014]["Tm"] == "MIA", "Week1"] = ""

#NYJ 2014
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "NYJ", "Week13"] = "MIA"
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "NYJ", "Week14"] = ""
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "NYJ", "Week16"] = "NE"
schedule_dfs[2014].loc[schedule_dfs[2014]["Tm"] == "NYJ", "Week17"] = ""


## Removing Away Game Attendance Values

In [147]:
#Adding Capacity column to Schedule DFs

for year in range(2013,2024):
    if year != 2020:
        schedule_dfs[year].insert(0, "Capacity", capacity_dfs[year]["Capacity"])

In [148]:
schedule_dfs[2013].head()

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17
0,63400,ARI,,DET,,,CAR,,SEA,ATL,,HOU,,IND,,LAR,,,SF
1,71228,ATL,,LAR,,NE,NYJ,,TB,,,SEA,,NO,,,WSH,,CAR
2,71008,BAL,,CLE,HOU,,,GB,,,,CIN,,NYJ,PIT,MIN,,NE,
3,71608,BUF,NE,CAR,,BAL,,CIN,,,KC,,NYJ,,ATL,,,MIA,
4,74867,CAR,SEA,,NYG,,,,LAR,,ATL,,NE,,TB,,NYJ,NO,


In [149]:
capacity_dfs[2023].head()

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,63400,ARI,64693,62080,62915,71521,62651,71202,68781,63064,67919,62861,70247,62177,66436,0,63967,59978,69879,63197
1,71000,ATL,69597,69251,63803,85716,69461,69921,62620,68008,69715,62861,0,70166,74455,69496,70301,69221,61752,70072
2,71008,BAL,70136,66015,70038,67919,67272,61011,70989,63064,70753,70424,70997,70240,0,70492,68021,71812,71196,70355
3,71608,BUF,83345,70772,64291,70881,61273,70515,64628,70416,66965,70318,70603,69879,0,73639,70933,67678,70772,66292
4,74867,CAR,69597,72709,68699,72842,63648,65529,0,71505,72301,61200,73543,62493,62432,70009,70301,70875,67233,69010


In [150]:
#Function to remove away game attendance values
def remove_away_games(schedule_df, attendance_df):
    
    #Creating a True/False dataframe (True = Away, False = Home)
    away_games_df = schedule_df == ""
    array = np.where(away_games_df, np.nan, attendance_df)
    df = pd.DataFrame(array, columns = schedule_df.columns)
                      
    return df

for year in range(2013, 2024):
    if year != 2020:
        capacity_dfs[year] = remove_away_games(schedule_dfs[year], capacity_dfs[year])

capacity_dfs[2023].head()

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,63400,ARI,NaN,62080,62915,NaN,62651,NaN,NaN,63064,NaN,62861,NaN,62177,NaN,NaN,63967,NaN,NaN,63197
1,71000,ATL,69597,69251,NaN,NaN,69461,69921,NaN,NaN,69715,NaN,NaN,70166,NaN,69496,NaN,69221,NaN,NaN
2,71008,BAL,70136,NaN,70038,NaN,NaN,NaN,70989,NaN,70753,70424,70997,NaN,NaN,70492,NaN,NaN,71196,70355
3,71608,BUF,NaN,70772,NaN,70881,61273,70515,NaN,70416,NaN,70318,70603,NaN,NaN,NaN,70933,NaN,70772,NaN
4,74867,CAR,NaN,72709,NaN,72842,NaN,NaN,NaN,71505,72301,NaN,73543,NaN,NaN,NaN,70301,70875,NaN,69010


### Checking All Teams Have 8 Home Games

In [151]:
# Check to see if each team has 8 home games
for year in range(2013,2024):
    if year != 2020:
        print((capacity_dfs[year].iloc[:,2:].notna().sum(axis = 1) > 8).sum())

0
0
0
0
0
0
0
16
16
16


## Removing Instances Where all Attendance Values are Equal

In [152]:
#2015, Jets
filter = capacity_dfs[2015]["Tm"] == "NYJ"
capacity_dfs[2015][filter]

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17
24,82500,NYJ,78160,NaN,78160,NaN,NaN,78160,NaN,NaN,78160,78160,NaN,78160,NaN,78160,NaN,78160,NaN


In [110]:
filter = schedule_dfs[2015]["Tm"] == "NYJ"
schedule_dfs[2015][filter]

,Team,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17
24,NYJ,CLE,,PHI,,,WSH,,,JAX,BUF,,MIA,,TEN,,NE,


# Formating Data for Machine Learning

## Transposing Data

In [153]:
capacity_dfs[2023].head()

,Capacity,Tm,Week1,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,Week10,Week11,Week12,Week13,Week14,Week15,Week16,Week17,Week18
0,63400,ARI,NaN,62080,62915,NaN,62651,NaN,NaN,63064,NaN,62861,NaN,62177,NaN,NaN,63967,NaN,NaN,63197
1,71000,ATL,69597,69251,NaN,NaN,69461,69921,NaN,NaN,69715,NaN,NaN,70166,NaN,69496,NaN,69221,NaN,NaN
2,71008,BAL,70136,NaN,70038,NaN,NaN,NaN,70989,NaN,70753,70424,70997,NaN,NaN,70492,NaN,NaN,71196,70355
3,71608,BUF,NaN,70772,NaN,70881,61273,70515,NaN,70416,NaN,70318,70603,NaN,NaN,NaN,70933,NaN,70772,NaN
4,74867,CAR,NaN,72709,NaN,72842,NaN,NaN,NaN,71505,72301,NaN,73543,NaN,NaN,NaN,70301,70875,NaN,69010


In [154]:
pd.melt(capacity_dfs[2013], id_vars = ["Tm", "Capacity"], var_name = "Week", value_name = "Attendance")

,Tm,Capacity,Week,Attendance
0,ARI,63400,Week1,NaN
1,ATL,71228,Week1,NaN
2,BAL,71008,Week1,NaN
3,BUF,71608,Week1,69519
4,CAR,74867,Week1,73294
...,...,...,...,...
539,SEA,69000,Week17,68264
540,SF,69732,Week17,NaN
541,TB,69218,Week17,NaN
542,TEN,69143,Week17,69143


In [155]:
capacity_dfs_long = {}

for year in range(2013,2024):
    if year != 2020:
        capacity_dfs_long[year] = pd.melt(capacity_dfs[year], id_vars = ["Tm", "Capacity"], var_name = "Week", value_name = "Attendance")
        
        #Removing all Away Game Rows
        capacity_dfs_long[year].dropna(inplace = True)
        
        capacity_dfs_long[year]["Week"] = capacity_dfs_long[year]["Week"].str.replace("Week", "").astype("int")
        capacity_dfs_long[year]["Attendance"] = capacity_dfs_long[year]["Attendance"].astype("int")
        capacity_dfs_long[year]["Year"] = int(year)
       
        

capacity_dfs_long[2023].head()

,Tm,Capacity,Week,Attendance,Year
1,ATL,71000,1,69597,2023
2,BAL,71008,1,70136,2023
5,CHI,61500,1,62456,2023
7,CLE,67431,1,67919,2023
9,DEN,76125,1,76299,2023


## Combining All Years

In [156]:
capacity_df_all = pd.concat([capacity_dfs_long[2013],
                            capacity_dfs_long[2014],
                            capacity_dfs_long[2015],
                            capacity_dfs_long[2016],
                            capacity_dfs_long[2017],
                            capacity_dfs_long[2018],
                            capacity_dfs_long[2019],
                            capacity_dfs_long[2021],
                            capacity_dfs_long[2022],
                            capacity_dfs_long[2023]], axis = 0, ignore_index = True)

capacity_df_all

,Tm,Capacity,Week,Attendance,Year
0,BUF,71608,1,69519,2013
1,CAR,74867,1,73294,2013
2,CHI,61500,1,62213,2013
3,CLE,67431,1,71513,2013
4,DAL,80000,1,85348,2013
...,...,...,...,...,...
2602,NO,73208,18,70072,2023
2603,NYG,82500,18,75740,2023
2604,SF,68500,18,71624,2023
2605,TEN,69143,18,63744,2023


## Adding Win Total Column
#### Step 1: Scrape Weekly Matchup Data

In [157]:
# Define headers for the HTTP request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Function to extract the number of wins from the record, accounting for ties
def extract_wins(record):
    try:
        parts = record.split('-')
        wins = int(parts[0])  # Extract the number of wins
        return wins
    except ValueError:
        return 0

# Function to determine the winner of the game
def determine_winner(game):
    scores = game.find_all('div', class_='ScoreCell__Score')
    if len(scores) == 2:
        score_away = int(scores[0].text.strip())
        score_home = int(scores[1].text.strip())
        if score_away > score_home:
            return 'away'
        elif score_home > score_away:
            return 'home'
    return None

# Prepare a list to store the data
weekly_record_data = []

start_time = time.time()

#Looping through each year
for year in range(2013,2024):
    if year != 2020:
        # Determine the number of weeks for each year
        if year < 2021:
            weeks = range(1, 18)  # 17 weeks
        else:
            weeks = range(1, 19)  # 18 weeks

        for week in weeks:
            
            # Update the URL for the specific week and year
            url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
            
            # Make the HTTP request to the URL
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, "html.parser")          
            
            # Find all game elements
            games = soup.find_all('div', class_='Scoreboard__Row')
            
            # Loop over each game and extract the relevant information
            for game in games:
                # Extract competitors (both teams)
                competitors = game.find_all('li', class_='ScoreboardScoreCell__Item')
            
                # Check if there are exactly two teams in the game
                if len(competitors) == 2:
                    # Extract information for each team
                    away_team_tag = competitors[0].find('a', class_='AnchorLink')
                    away_team_abbr = away_team_tag.get('href').split('/')[5].upper() if away_team_tag else "Unknown"
                    away_team_record_tag = competitors[0].find('span', class_='ScoreboardScoreCell__Record')
                    away_team_record = away_team_record_tag.text.strip() if away_team_record_tag else '0-0'
                    away_team_wins = extract_wins(away_team_record)
            
                    home_team_tag = competitors[1].find('a', class_='AnchorLink')
                    home_team_abbr = home_team_tag.get('href').split('/')[5].upper() if home_team_tag else "Unknown"
                    home_team_record_tag = competitors[1].find('span', class_='ScoreboardScoreCell__Record')
                    home_team_record = home_team_record_tag.text.strip() if home_team_record_tag else '0-0'
                    home_team_wins = extract_wins(home_team_record)
            
                    # Determine the winner of the game
                    winner = determine_winner(game)
            
                    # Adjust the win counts based on the game result
                    if winner == 'away':
                        away_team_wins -= 1
                    elif winner == 'home':
                        home_team_wins -= 1
            
                    # Append the data to the list
                    weekly_record_data.append([
                        year,
                        week,
                        away_team_abbr,
                        away_team_wins,
                        home_team_abbr,
                        home_team_wins
                    ])

# Create a DataFrame from the collected data
weekly_record_df = pd.DataFrame(weekly_record_data, columns=[
    'Year', 'Week', 'Away Team', 'Away Team Wins', 'Home Team', 'Home Team Wins'
])

# Display the DataFrame
print(weekly_record_df)

end_time = time.time()
total_time = end_time - start_time
print(f"Time to Scrape Weekly Record Data 2013-2023: {total_time:.2f} seconds")

      Year  Week Away Team  Away Team Wins Home Team  Home Team Wins
0     2013     1       BAL               0       DEN               0
1     2013     1        NE               0       BUF               0
2     2013     1       CIN               0       CHI               0
3     2013     1       MIA               0       CLE               0
4     2013     1       MIN               0       DET               0
...    ...   ...       ...             ...       ...             ...
2605  2023    18       SEA               8       ARI               4
2606  2023    18        KC              10       LAC               5
2607  2023    18       LAR               9        SF              12
2608  2023    18       DAL              11       WSH               4
2609  2023    18       BUF              10       MIA              11

[2610 rows x 6 columns]
Time to Scrape Weekly Record Data 2013-2023: 306.31 seconds


In [158]:
weekly_record_df["Home Team Wins"].value_counts()

Home Team Wins
0     354
1     349
2     335
3     332
4     288
5     260
6     183
7     176
8     123
9      85
10     58
11     36
12     23
13      7
14      1
Name: count, dtype: int64

#### Step 2: Clean Matchup Data

In [159]:
# Deleting Duplicate Row for Postponed Game (NYJ vs. BUF 2014)
weekly_record_df.drop(430, axis = 0, inplace = True)
weekly_record_df[(weekly_record_df["Home Team"] == "BUF") & (weekly_record_df["Year"] == 2014)]

,Year,Week,Away Team,Away Team Wins,Home Team,Home Team Wins
273,2014,2,MIA,1,BUF,1
289,2014,3,LAC,1,BUF,2
333,2014,6,NE,3,BUF,3
348,2014,7,MIN,2,BUF,3
391,2014,10,KC,5,BUF,5
431,2014,12,NYJ,2,BUF,5
436,2014,13,CLE,7,BUF,6
467,2014,15,GB,10,BUF,7


In [160]:
# Deleting Duplicate Row for Postponed Game (TB vs. MIA 2017)
weekly_record_df.drop(1038, axis = 0, inplace = True)
weekly_record_df[(weekly_record_df["Home Team"] == "MIA") & (weekly_record_df["Year"] == 2017)]

,Year,Week,Away Team,Away Team Wins,Home Team,Home Team Wins
1074,2017,4,NO,1,MIA,1
1094,2017,5,TEN,2,MIA,1
1124,2017,7,NYJ,3,MIA,3
1156,2017,9,LV,3,MIA,4
1176,2017,11,TB,3,MIA,4
1208,2017,13,DEN,3,MIA,4
1233,2017,14,NE,10,MIA,5
1277,2017,17,BUF,8,MIA,6


In [161]:
#Changing Washington Commanders Abbreviation from WSH to WAS
weekly_record_df.loc[weekly_record_df["Home Team"] == "WSH", "Home Team"] = "WAS"

In [162]:
# Check for Missing Values
weekly_record_df.loc[weekly_record_df["Home Team Wins"].isna()]

,Year,Week,Away Team,Away Team Wins,Home Team,Home Team Wins


#### Step 3: Merge Weekly Matchup Data with Main Data

In [163]:
#
capacity_df_all2 = pd.merge(capacity_df_all, weekly_record_df, left_on = ["Tm", "Year", "Week"], right_on = ["Home Team", "Year", "Week"], how = "left")
capacity_df_all2.drop(columns = {"Away Team", "Home Team"}, inplace = True)
capacity_df_all2["Away Team Wins"] = capacity_df_all2["Away Team Wins"].astype(int)
capacity_df_all2["Home Team Wins"] = capacity_df_all2["Home Team Wins"].astype(int)
capacity_df_all2.tail()

,Tm,Capacity,Week,Attendance,Year,Away Team Wins,Home Team Wins
2602,NO,73208,18,70072,2023,7,8
2603,NYG,82500,18,75740,2023,11,5
2604,SF,68500,18,71624,2023,9,12
2605,TEN,69143,18,63744,2023,9,5
2606,WAS,67717,18,63645,2023,11,4


In [164]:
# Check for Missing Values
capacity_df_all2.loc[capacity_df_all2["Home Team Wins"].isna()]

,Tm,Capacity,Week,Attendance,Year,Away Team Wins,Home Team Wins


## Marking Primetime Games

### Scraping Primetime Game Schedules

In [165]:
def scrape_primetime_schedule(url):
    # Send a GET request to the webpage
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all schedule entries
        schedule_items = soup.find_all('div', class_='nfl-c-matchup-tile')

        # List to store scraped data
        data = []

        # Loop through each schedule entry
        for item in schedule_items:
            # Extract week number
            week_info = item.find('span', class_='nfl-c-matchup-tile__week-name')
            if week_info:
                week_number_text = week_info.get_text(strip=True)
                # Clean up non-numeric characters and convert to integer
                week_number_text_clean = re.sub(r'\D', '', week_number_text)  # Remove non-digit characters
                week_number = int(week_number_text_clean) if week_number_text_clean else "Unknown Week"
            else:
                week_number = "Unknown Week"
            
            # Extract day of the week
            day_info = item.find('span', class_='nfl-c-matchup-tile__week-day')
            if day_info:
                day_of_week = day_info.get_text(strip=True).split('·')[0].strip()  # Extract day and remove extra characters
            else:
                day_of_week = "Unknown Day"
            
            # Extract home team
            match_strip = item.find('div', class_='nfl-c-matchup-strip')
            team_abbreviations = match_strip.find_all('span', class_='nfl-c-matchup-strip__team-abbreviation') if match_strip else []
            if len(team_abbreviations) >= 2:
                home_team = team_abbreviations[1].get_text(strip=True)  # The second abbreviation is typically the home team
            else:
                home_team = "Unknown Team"

            # Append data to the list
            data.append({
                'Week Number': week_number,
                'Home Team': home_team,
                'Day of Week': day_of_week
            })
        
        # Convert the data to a pandas DataFrame
        df = pd.DataFrame(data)
        
        # Reset index to start at 0
        df.index = range(len(df))
        
        return df
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None
        
start_time = time.time()

snf = {}
for year in range(2013,2024):
    if year != 2020:
        url = f"https://www.nfl.com/schedules/sunday-night-football/{year}"
        snf[year] = scrape_primetime_schedule(url)
        snf[year]["Year"] = year

mnf = {}
for year in range(2013,2024):
    if year != 2020:
        url = f"https://www.nfl.com/schedules/monday-night-football/{year}"
        mnf[year] = scrape_primetime_schedule(url)
        mnf[year]["Year"] = year

tnf = {}
for year in range(2013,2024):
    if year != 2020:
        url = f"https://www.nfl.com/schedules/thursday-night-football/{year}"
        tnf[year] = scrape_primetime_schedule(url)
        tnf[year]["Year"] = year


end_time = time.time()
total_time = end_time - start_time
print(f"Time to Scrape Primetime Game Data 2013-2023: {total_time:.2f} seconds")

Time to Scrape Primetime Game Data 2013-2023: 73.40 seconds


In [168]:
# Stacking all Primetime Dataframes

snf_combined = pd.concat(snf.values(), ignore_index = True)
mnf_combined = pd.concat(tnf.values(), ignore_index = True)
tnf_combined = pd.concat(mnf.values(), ignore_index = True)

primetime_df = pd.concat([snf_combined, mnf_combined, tnf_combined], ignore_index = True)

#Converting Week Column from float to int
primetime_df["Week Number"] = primetime_df["Week Number"].astype(int)

primetime_df

,Year,Week Number,Home Team,Day of Week
0,2017,1,NE,Thu
1,2017,1,DAL,Sun
2,2018,1,PHI,Thu
3,2018,1,GB,Sun
4,2018,2,DAL,Sun
...,...,...,...,...
274,2023,15,SEA,Mon
275,2023,16,SF,Mon
276,2023,17,DAL,Sat
277,2023,18,BAL,Sat


In [169]:
capacity_df_all3 = pd.merge(capacity_df_all2, primetime_df, left_on = ["Tm", "Week", "Year"], right_on = ["Home Team", "Week Number", "Year"], how = "left")
capacity_df_all3.loc[capacity_df_all3["Day of Week"].isna(), "Time"] = 0
capacity_df_all3.loc[capacity_df_all3["Day of Week"] == "Sun", "Time"] = 1
capacity_df_all3.loc[capacity_df_all3["Day of Week"] == "Mon", "Time"] = 2
capacity_df_all3.loc[capacity_df_all3["Day of Week"] == "Thu", "Time"] = 3
capacity_df_all3.loc[capacity_df_all3["Day of Week"] == "Fri", "Time"] = 4
capacity_df_all3.loc[capacity_df_all3["Day of Week"] == "Sat", "Time"] = 4
capacity_df_all3["Time"] = capacity_df_all3["Time"].astype(int)
capacity_df_all3.drop(columns = {"Week Number", "Home Team", "Day of Week"}, inplace = True)
capacity_df_all = capacity_df_all3

capacity_df_all

,Tm,Capacity,Week,Attendance,Year,Away Team Wins,Home Team Wins,Time
0,BUF,71608,1,69519,2013,0,0,0
1,CAR,74867,1,73294,2013,0,0,0
2,CHI,61500,1,62213,2013,0,0,0
3,CLE,67431,1,71513,2013,0,0,0
4,DAL,80000,1,85348,2013,0,0,0
...,...,...,...,...,...,...,...,...
2602,NO,73208,18,70072,2023,7,8,0
2603,NYG,82500,18,75740,2023,11,5,0
2604,SF,68500,18,71624,2023,9,12,0
2605,TEN,69143,18,63744,2023,9,5,0


In [ ]:
# Thanksgiving Games --> Holiday

## Marking International Games

In [170]:
# Setting All Games to 0
capacity_df_all["International"] = 0

# Inputting 1's for International Games
capacity_df_all.loc[(capacity_df_all["Tm"] == "MIN") & (capacity_df_all["Year"] == 2013) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2013) & (capacity_df_all["Week"] == 8), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "LV") & (capacity_df_all["Year"] == 2014) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "ATL") & (capacity_df_all["Year"] == 2014) & (capacity_df_all["Week"] == 8), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2014) & (capacity_df_all["Week"] == 10), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "MIA") & (capacity_df_all["Year"] == 2015) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2015) & (capacity_df_all["Week"] == 7), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "KC") & (capacity_df_all["Year"] == 2015) & (capacity_df_all["Week"] == 8), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2016) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "LAR") & (capacity_df_all["Year"] == 2016) & (capacity_df_all["Week"] == 7), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "CIN") & (capacity_df_all["Year"] == 2016) & (capacity_df_all["Week"] == 8), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2017) & (capacity_df_all["Week"] == 3), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "MIA") & (capacity_df_all["Year"] == 2017) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "LAR") & (capacity_df_all["Year"] == 2017) & (capacity_df_all["Week"] == 7), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "CLE") & (capacity_df_all["Year"] == 2017) & (capacity_df_all["Week"] == 8), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "LV") & (capacity_df_all["Year"] == 2018) & (capacity_df_all["Week"] == 6), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "LAC") & (capacity_df_all["Year"] == 2018) & (capacity_df_all["Week"] == 7), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2018) & (capacity_df_all["Week"] == 8), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "LV") & (capacity_df_all["Year"] == 2019) & (capacity_df_all["Week"] == 5), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "TB") & (capacity_df_all["Year"] == 2019) & (capacity_df_all["Week"] == 6), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "LAR") & (capacity_df_all["Year"] == 2019) & (capacity_df_all["Week"] == 8), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2019) & (capacity_df_all["Week"] == 9), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "ATL") & (capacity_df_all["Year"] == 2021) & (capacity_df_all["Week"] == 5), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2021) & (capacity_df_all["Week"] == 6), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "NO") & (capacity_df_all["Year"] == 2022) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "GB") & (capacity_df_all["Year"] == 2022) & (capacity_df_all["Week"] == 5), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2022) & (capacity_df_all["Week"] == 8), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Year"] == 2023) & (capacity_df_all["Week"] == 4), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "BUF") & (capacity_df_all["Year"] == 2023) & (capacity_df_all["Week"] == 5), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "TEN") & (capacity_df_all["Year"] == 2023) & (capacity_df_all["Week"] == 6), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "LV") & (capacity_df_all["Year"] == 2016) & (capacity_df_all["Week"] == 11), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "LV") & (capacity_df_all["Year"] == 2017) & (capacity_df_all["Week"] == 11), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "LAC") & (capacity_df_all["Year"] == 2019) & (capacity_df_all["Week"] == 11), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "ARI") & (capacity_df_all["Year"] == 2022) & (capacity_df_all["Week"] == 11), "International"] = 1

capacity_df_all.loc[(capacity_df_all["Tm"] == "TB") & (capacity_df_all["Year"] == 2022) & (capacity_df_all["Week"] == 10), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "KC") & (capacity_df_all["Year"] == 2023) & (capacity_df_all["Week"] == 9), "International"] = 1
capacity_df_all.loc[(capacity_df_all["Tm"] == "NE") & (capacity_df_all["Year"] == 2023) & (capacity_df_all["Week"] == 10), "International"] = 1

In [171]:
capacity_df_all.loc[(capacity_df_all["International"] == 1)].count()

Tm                37
Capacity          37
Week              37
Attendance        37
Year              37
Away Team Wins    37
Home Team Wins    37
Time              37
International     37
dtype: int64

### Adjusting Capacity for International Games

In [173]:
# London Games
# Wembley Stadium Max Capacity: 86,000
# Twickenham Stadium Max Capacity: 75,000
# Tottenham Hotspur Stadium Max Capacity: 62,850

# 2013
capacity_df_all.loc[(capacity_df_all["Year"] == 2013) & (capacity_df_all["Tm"] == "MIN") & (capacity_df_all["Week"] == 4), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2013) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000

# 2014
capacity_df_all.loc[(capacity_df_all["Year"] == 2014) & (capacity_df_all["Tm"] == "LV") & (capacity_df_all["Week"] == 4), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2014) & (capacity_df_all["Tm"] == "ATL") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2014) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 10), "Capacity"] = 86000

# 2015
capacity_df_all.loc[(capacity_df_all["Year"] == 2015) & (capacity_df_all["Tm"] == "MIA") & (capacity_df_all["Week"] == 4), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2015) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 7), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2015) & (capacity_df_all["Tm"] == "KC") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000

# 2016
capacity_df_all.loc[(capacity_df_all["Year"] == 2016) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 4), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2016) & (capacity_df_all["Tm"] == "LAR") & (capacity_df_all["Week"] == 7), "Capacity"] = 75000
capacity_df_all.loc[(capacity_df_all["Year"] == 2016) & (capacity_df_all["Tm"] == "CIN") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000

# 2017
capacity_df_all.loc[(capacity_df_all["Year"] == 2017) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 3), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2017) & (capacity_df_all["Tm"] == "MIA") & (capacity_df_all["Week"] == 4), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2017) & (capacity_df_all["Tm"] == "LAR") & (capacity_df_all["Week"] == 7), "Capacity"] = 75000
capacity_df_all.loc[(capacity_df_all["Year"] == 2017) & (capacity_df_all["Tm"] == "CLE") & (capacity_df_all["Week"] == 8), "Capacity"] = 75000

# 2018
capacity_df_all.loc[(capacity_df_all["Year"] == 2018) & (capacity_df_all["Tm"] == "LV") & (capacity_df_all["Week"] == 6), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2018) & (capacity_df_all["Tm"] == "LAC") & (capacity_df_all["Week"] == 7), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2018) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000

# 2019
capacity_df_all.loc[(capacity_df_all["Year"] == 2019) & (capacity_df_all["Tm"] == "LV") & (capacity_df_all["Week"] == 5), "Capacity"] = 62850
capacity_df_all.loc[(capacity_df_all["Year"] == 2019) & (capacity_df_all["Tm"] == "TB") & (capacity_df_all["Week"] == 6), "Capacity"] = 62850
capacity_df_all.loc[(capacity_df_all["Year"] == 2019) & (capacity_df_all["Tm"] == "LAR") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2019) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 9), "Capacity"] = 86000

# 2021
capacity_df_all.loc[(capacity_df_all["Year"] == 2021) & (capacity_df_all["Tm"] == "ATL") & (capacity_df_all["Week"] == 5), "Capacity"] = 62850
capacity_df_all.loc[(capacity_df_all["Year"] == 2021) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 6), "Capacity"] = 62850

# 2022
capacity_df_all.loc[(capacity_df_all["Year"] == 2022) & (capacity_df_all["Tm"] == "NO") & (capacity_df_all["Week"] == 4), "Capacity"] = 62850
capacity_df_all.loc[(capacity_df_all["Year"] == 2022) & (capacity_df_all["Tm"] == "GB") & (capacity_df_all["Week"] == 5), "Capacity"] = 62850
capacity_df_all.loc[(capacity_df_all["Year"] == 2022) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 8), "Capacity"] = 86000

# 2023
capacity_df_all.loc[(capacity_df_all["Year"] == 2023) & (capacity_df_all["Tm"] == "JAX") & (capacity_df_all["Week"] == 4), "Capacity"] = 86000
capacity_df_all.loc[(capacity_df_all["Year"] == 2023) & (capacity_df_all["Tm"] == "BUF") & (capacity_df_all["Week"] == 5), "Capacity"] = 62850
capacity_df_all.loc[(capacity_df_all["Year"] == 2023) & (capacity_df_all["Tm"] == "TEN") & (capacity_df_all["Week"] == 6), "Capacity"] = 62850

# Mexico Games
# Estadio Azteca Max Capacity: 83,000
capacity_df_all.loc[(capacity_df_all["Year"] == 2016) & (capacity_df_all["Tm"] == "LV") & (capacity_df_all["Week"] == 11), "Capacity"] = 83000
capacity_df_all.loc[(capacity_df_all["Year"] == 2017) & (capacity_df_all["Tm"] == "LV") & (capacity_df_all["Week"] == 11), "Capacity"] = 83000
capacity_df_all.loc[(capacity_df_all["Year"] == 2019) & (capacity_df_all["Tm"] == "LAC") & (capacity_df_all["Week"] == 11), "Capacity"] = 83000
capacity_df_all.loc[(capacity_df_all["Year"] == 2022) & (capacity_df_all["Tm"] == "ARI") & (capacity_df_all["Week"] == 11), "Capacity"] = 83000

# Germany Games
# Alianz Arena Max Capacity: 70,000
# Deutsche Bank Park Max Capacity: 50,500
capacity_df_all.loc[(capacity_df_all["Year"] == 2022) & (capacity_df_all["Tm"] == "TB") & (capacity_df_all["Week"] == 10), "Capacity"] = 70000
capacity_df_all.loc[(capacity_df_all["Year"] == 2023) & (capacity_df_all["Tm"] == "KC") & (capacity_df_all["Week"] == 9), "Capacity"] = 50000
capacity_df_all.loc[(capacity_df_all["Year"] == 2023) & (capacity_df_all["Tm"] == "NE") & (capacity_df_all["Week"] == 10), "Capacity"] = 50000


## Final Checks

In [175]:
capacity_df_all.sort_values(by = "Attendance")

,Tm,Capacity,Week,Attendance,Year,Away Team Wins,Home Team Wins,Time,International
2305,CIN,65515,17,0,2022,13,12,2,0
1175,LAC,27000,11,25015,2017,5,3,0,0
1208,LAC,27000,13,25320,2017,0,5,0,0
1449,LAC,27000,12,25343,2018,2,7,0,0
1575,LAC,27000,3,25349,2019,1,1,0,0
...,...,...,...,...,...,...,...,...,...
2533,DAL,80000,14,93752,2023,10,9,1,0
2292,DAL,80000,16,93754,2022,13,10,0,0
2174,DAL,80000,8,93767,2022,3,5,0,0
2068,DAL,80000,1,93797,2022,0,0,1,0


In [176]:
#Removing Week 17 2022 "Damar Hamlin Game"
capacity_df_all.drop(capacity_df_all.loc[(capacity_df_all["Tm"] == "CIN") &
                                         (capacity_df_all["Year"] == 2022) &
                                         (capacity_df_all["Week"] == 17)].index, inplace = True)

#Removing Week 1 2017 "Hurricane Ida (Game played in Jacksonville)"
capacity_df_all.drop(capacity_df_all.loc[(capacity_df_all["Tm"] == "NO") &
                                         (capacity_df_all["Year"] == 2021) &
                                         (capacity_df_all["Week"] == 1)].index, inplace = True)

capacity_df_all.sort_values(by = "Attendance")

,Tm,Capacity,Week,Attendance,Year,Away Team Wins,Home Team Wins,Time,International
1175,LAC,27000,11,25015,2017,5,3,0,0
1208,LAC,27000,13,25320,2017,0,5,0,0
1449,LAC,27000,12,25343,2018,2,7,0,0
1575,LAC,27000,3,25349,2019,1,1,0,0
1287,LAC,27000,1,25351,2018,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2533,DAL,80000,14,93752,2023,10,9,1,0
2292,DAL,80000,16,93754,2022,13,10,0,0
2174,DAL,80000,8,93767,2022,3,5,0,0
2068,DAL,80000,1,93797,2022,0,0,1,0


# Writing Data to CSV file for Machine Learning

In [178]:
capacity_df_all.to_csv("Data Files/Cleaned_Data_for_ML.csv", index = False)